In [2]:
!python -V

Python 3.9.21


In [3]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db") 
mlflow.set_experiment("ny-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1', creation_time=1746683774913, experiment_id='1', last_update_time=1746683774913, lifecycle_stage='active', name='ny-taxi-experiment', tags={}>

In [4]:
#install pyarrow to read parquet file

#!pip install pyarrow

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error



In [6]:
#!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet 

In [7]:
#!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet 

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename,engine='pyarrow')

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:


df_train = read_dataframe('./data/green_tripdata_2021-02.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-01.parquet')


In [10]:
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)


In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)


50.01983345993515

In [14]:
#Upload model to the model folder dv is data set vectoried and ltr is model
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)


In [15]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Shakti")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-02.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-01.parque")
    alpha = 0.1
    mlflow.log_param("alpha",alpha)



    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)

    rmse= mean_squared_error(y_val, y_pred)
    mlflow.log_param("rmse",rmse)
    mlflow.log_artifact("models/lin_reg.bin", artifact_path="models_pickle")





In [16]:
import xgboost as xgb

#hyperopt used to find best parameters
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:


def objective(params):
    with mlflow.start_run():
         mlflow.set_tag("model", "xgboost")
         mlflow.log_params(params)
         booster = xgb.train(
              params=params,
              dtrain=train,
              num_boost_round=1000,
              evals=[(valid, "validation")],
              early_stopping_rounds=50
         )

         y_pred = booster.predict(valid)
         rmse = mean_squared_error(y_val, y_pred) ** 0.5
         mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}




search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed':42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials())



In [ ]:
#Choose best parameters from mlflow by uisng lowest rmse and less training time  
#logging best params
import mlflow.xgboost


with mlflow.start_run():

    params = {
        'learning_rate': 0.09064205157889488,
        'max_depth': 22,
        'min_child_weight': 1.0629059550227644,
        'objective': 'reg:linear',  # consider using 'reg:squarederror' if using newer XGBoost
        'reg_alpha': 0.13507272239403997,
        'reg_lambda': 0.01337017283417198,
        'seed': 42
    }

#Autolog to get all logs without much code


    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)
    booster=xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid,"validation")],
            early_stopping_rounds=50
            
        )    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred) ** 0.5
    mlflow.log_metric("rmse", rmse)


    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster,artifact_path="models_mlflow")




/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:25:33] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.79321
[1]	validation-rmse:10.07803
[2]	validation-rmse:9.45202
[3]	validation-rmse:8.90457
[4]	validation-rmse:8.43150
[5]	validation-rmse:8.02212
[6]	validation-rmse:7.67306
[7]	validation-rmse:7.37425
[8]	validation-rmse:7.12181
[9]	validation-rmse:6.90909


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:25:42] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/05/10 17:25:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
